In [1]:
import os
import argparse
import json
import cv2
import keras
import numpy as np
import tensorflow as tf
from utils.utils import get_yolo_boxes
from utils.bbox import draw_boxes
from keras.models import load_model
from tqdm import tqdm

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
infer_model = load_model('./snapshots/person_trassir/yolo3_model.h5')

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:270: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [3]:
for layer in infer_model.layers:
    layer.is_trainable = False

In [4]:
C4 = infer_model.get_layer('conv_80').output
C3 = infer_model.get_layer('conv_92').output
C2 = infer_model.get_layer('conv_104').output

roi_input = keras.layers.Input(shape=(32, 4), name='input_2')

print(C2.shape)
print(C3.shape)
print(C4.shape)

(?, ?, ?, 256)
(?, ?, ?, 512)
(?, ?, ?, 1024)


In [6]:
from masknet import *

masknet_model = create_model(image_size=(608, 608, 3))
masknet_model.inputs

[<tf.Tensor 'input_C2:0' shape=(?, 152, 152, 256) dtype=float32>,
 <tf.Tensor 'input_C3:0' shape=(?, 76, 76, 512) dtype=float32>,
 <tf.Tensor 'input_C4:0' shape=(?, 38, 38, 1024) dtype=float32>,
 <tf.Tensor 'input_rois:0' shape=(?, 32, 4) dtype=float32>]

In [7]:
x = masknet_model([C2, C3, C4, roi_input])
common_model = Model(inputs=[infer_model.inputs[0], roi_input], outputs=x)

In [8]:
predicted = common_model.predict([np.zeros((1, 416, 416, 3)), np.zeros((1, 32, 4))])
predicted.shape

(1, 32, 28, 28, 1)

In [9]:
common_model.layers[-1].layers[14].image_shape

(608, 608, 3)

In [16]:
a = infer_model.input.get_shape()

NoneType